In [3]:
import numpy as np

Ejercicio 1 - Implementar descompLU: 

In [10]:
def descompLU(A):
    # Como no hay pivoteo, si A[0][0] es 0, debe tirar error
    if A[0][0] == 0:
        print('ERROR: el elemento A[0][0] es 0')
        return np.eye(A.shape[0]), A
    # Caso Base
    if A.shape == (1,1):
        return 1, A[0][0] 

    # Busco los datos para armar mis matrices L y U
    u11 = A[0][0]
    U12 = A[0][1:]
    L21 = A[1:,0] / u11
    L22U22 = A[1:,1:] - np.outer(L21, U12)

    # empiezo a armar mi L, U de cada vuelta de la recursión
    L22, U22 = descompLU(L22U22)

    U = np.zeros(A.shape)
    U[0][0] = u11
    U[0][1:] = U12
    U[1:, 1:] = U22

    L = np.eye(A.shape[0])
    L[1:,0] = L21
    L[1:, 1:] = L22

    return L, U

Ejemplo de uso:

In [11]:
A = np.array([[1,2,3], [4,5,6], [7,8,9]])
L, U = descompLU(A)

print(L,"\n")
print(U)

ERROR: el elemento A[0][0] es 0
[[1. 0. 0.]
 [4. 1. 0.]
 [7. 2. 1.]] 

[[ 1.  2.  3.]
 [ 0. -3. -6.]
 [ 0.  0.  0.]]


In [ ]:
def resolverLU(A, b):
    L, U = descompLU(A)
    